In [1]:
!pip install gurobipy

zsh:1: command not found: pip


In [2]:
!pip install pandas

zsh:1: command not found: pip


In [ ]:
# Integer Linear Programs
# Sequence-based Model
import pandas as pd
import gurobipy as gp
from google.colab import files

uploaded = files.upload()


# Create an environment with your WLS license
params = {
"WLSACCESSID": '51902970-d408-4dfc-bfdc-50d992f0ff02',
"WLSSECRET": 'cb545298-cbf0-4c76-b096-6540d3314e04',
"LICENSEID": 2435093,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
m = gp.Model(env=env)


# Read the excel file and extract the Test data sheet
data = pd.read_excel('rjsumCj.xlsx', sheet_name='Test data')

# Extract job information from the data
jobs = data['job'].tolist()
#release_dates = data['release date d_j'].tolist()
rd = data['release date d_j'].tolist()
#processing_lengths = data['processing length p_j'].tolist()
p = data['processing length p_j'].tolist()

# Initialize the total number of jobs (n)
#n = len(jobs)
n = 22
M = 10000
# Add decision variables
x = m.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
C = m.addVars(n, vtype=gp.GRB.INTEGER, name="C")

# Set the objective function
m.setObjective(C.sum(), gp.GRB.MINIMIZE)

# Add constraints for job processing

# constraint1
for i in range(n):
  for j in range(n):
    if i != j:
      m.addConstr(x[i,j]+x[j,i]==1)
#2
for j in range(n):
  m.addConstr(C[j]-(rd[j]+p[j])>=0)
#3
for i in range(n):
  for j in range(n):
    if i != j:
      m.addConstr(C[j]-p[j]+(1-x[i,j]) * M >= C[i])

# limit
m.Params.TimeLimit = 3600

# Optimize the model
m.optimize()

# Print the optimal schedule
print("Optimal Schedule:")

print("\nobjective:", m.objVal)
print("Runtime:", m.Runtime)
print("Gap:", m.getAttr('MIPGap'))

Saving rjsumCj.xlsx to rjsumCj.xlsx
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2435093
Academic license - for non-commercial use only - registered to coco.chen.mg12@nycu.edu.tw
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to coco.chen.mg12@nycu.edu.tw
Optimize a model with 946 rows, 506 columns and 2332 nonzeros
Model fingerprint: 0xc52ff566
Variable types: 0 continuous, 506 integer (484 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 484 rows and 253 columns
Presolve time: 0.00s
Presolved: 462 rows, 253 columns, 1386 nonzeros
Variable types: 0 continu

In [ ]:
# Position-based Model

import pandas as pd
import gurobipy as gp
from gurobipy import quicksum
from google.colab import files

uploaded = files.upload()

# Create an environment with your WLS license
params = {
"WLSACCESSID": '51902970-d408-4dfc-bfdc-50d992f0ff02',
"WLSSECRET": 'cb545298-cbf0-4c76-b096-6540d3314e04',
"LICENSEID": 2435093,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
m2 = gp.Model(env=env)


# Read the excel file and extract the Test data sheet
data = pd.read_excel('rjsumCj.xlsx', sheet_name='Test data')

# Extract job information from the data
jobs = data['job'].tolist()
rd = data['release date d_j'].tolist()
p = data['processing length p_j'].tolist()

# Initialize the total number of jobs (n)
# set n
n = 38
# Add decision variables
x = m2.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
C = m2.addVars(n, vtype=gp.GRB.INTEGER, name="C")

# Set the objective function
m2.setObjective(C.sum(), gp.GRB.MINIMIZE)

# Add constraints for job processing
# 2-1
for j in range(n):
      m2.addConstr(sum(x[j, k] for k in range(n)) == 1)

# 2-2 (7)
for j in range(n):
      m2.addConstr(sum(x[k, j] for k in range(n)) == 1)
#8
#for j in range(n):
#      m.addConstrs(C[0]-sum(x[j,0]*(rd[j]+p[j])) >= 0)
expr_list = [x[j, 0] * (rd[j] + p[j]) for j in range(n)]
m2.addConstr(C[0] - gp.quicksum(expr_list) >= 0)

#9
#for j in range(n):
#  for k in range(1,n):
#    m.addConstr(C[k]-C[k-1]-sum(x[j,k]*p[j]) >= 0)

for k in range(1,n):
  expr_list = sum(x[j, k] * p[j] for j in range(n))
  m2.addConstr((C[k] - C[k - 1] - expr_list)>= 0)

#10
#for j in range(n):
#  for k in range(1,n):
#    m.addConstr(C[k]-sum(x[j,k]*(rd[j]+p[j])) >=0)
for k in range(1,n):
  expr_list = sum(x[j, k] * (rd[j] + p[j]) for j in range(n))
  m2.addConstr((C[k] - expr_list) >= 0)

# limit
m2.Params.TimeLimit = 3600

# Optimize the model
m2.optimize()

# Print the optimal schedule
print("\nobjective:", m2.objVal)
print("Runtime:", m2.Runtime)
print("Gap:", m2.getAttr('MIPGap'))

Saving rjsumCj.xlsx to rjsumCj (1).xlsx
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2435093
Academic license - for non-commercial use only - registered to coco.chen.mg12@nycu.edu.tw
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to coco.chen.mg12@nycu.edu.tw
Optimize a model with 151 rows, 1482 columns and 5850 nonzeros
Model fingerprint: 0x30a98cb0
Variable types: 0 continuous, 1482 integer (1444 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.05s
Presolved: 151 rows, 1482 columns, 5698 nonzeros
Variable types: 0 continuous, 1482 integer (1444 binary)
Fo

In [ ]:
#Time-indexed Model
import pandas as pd
import gurobipy as gp
from gurobipy import quicksum
from google.colab import files

uploaded = files.upload()


# Create an environment with your WLS license
params = {
"WLSACCESSID": '51902970-d408-4dfc-bfdc-50d992f0ff02',
"WLSSECRET": 'cb545298-cbf0-4c76-b096-6540d3314e04',
"LICENSEID": 2435093,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
m3 = gp.Model(env=env)


# Read the excel file and extract the Test data sheet
data = pd.read_excel('rjsumCj.xlsx', sheet_name='Test data')

# Extract job information from the data
jobs = data['job'].tolist()
rd = data['release date d_j'].tolist()
p = data['processing length p_j'].tolist()

# Initialize the total number of jobs (n)
# set n
n = 50
# Add decision variables
T = max(rd)+sum(p)
x = m3.addVars(n, T+1, vtype=gp.GRB.BINARY, name="x")
C = m3.addVars(n, vtype=gp.GRB.INTEGER, name="C")


# Set the objective function
m3.setObjective(C.sum(), gp.GRB.MINIMIZE)

# Add constraints for job processing

# constraint

#13
for j in range(n):
      m3.addConstr(sum(x[j,t]  for t in range(rd[j],T+1))== 1)

#14
for j in range(n):
  for t in range(min(rd),T+1):
      m3.addConstr(sum(x[j,s]for s in range(max(t-p[j]+1,rd[j]),t+1))<= 1)

#15
for j in range(n):
    m3.addConstr(C[j]-p[j]>=sum(x[j,t]*t for t in range(rd[j],T+1)))

#16
for j in range(n):
  for t in range(j,rd[j]):
    m3.addConstr(x[j,t]== 0)

m3.addConstrs(sum(x[j, t] for t in range(rd[j], T + 1)) == 1 for j in range(n))

# limit
m3.Params.TimeLimit = 3600

# Optimize the model
m3.optimize()

# Print the optimal schedule
print("\nobjective:", m3.objVal)
print("Runtime:", m3.Runtime)
print("Gap:", m3.getAttr('MIPGap'))

Saving rjsumCj.xlsx to rjsumCj (28).xlsx
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2435093
Academic license - for non-commercial use only - registered to coco.chen.mg12@nycu.edu.tw
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to coco.chen.mg12@nycu.edu.tw
Optimize a model with 126004 rows, 105350 columns and 2297412 nonzeros
Model fingerprint: 0x49105ec6
Variable types: 0 continuous, 105350 integer (105300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 64761.000000
Presolve removed 126004 rows and 105350 columns
Presolve time: 1.71